In [1]:
from array import array
import time
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from numpy import random, arange
from scipy.signal import butter, sosfilt, sosfreqz, lfilter
import datetime
import random
from tkinter import *  
from tkinter.ttk import Progressbar  
from tkinter import ttk  
from tkinter import filedialog  
import sqlalchemy
from scipy import interpolate
import sys
import mysql
import mysql.connector
import pickle
import warnings
import xgboost as xgb
import lightgbm as lgbm
import catboost as catb
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
warnings.simplefilter('ignore')
import pathlib
import os



In [3]:
from forces import vagon_1
from forces import vagon_2
from forces import vagon_3
from forces import vagon_4
from tensions import krom_tension
from tensions import f_shpal
from tensions import f_ballast_opzp

In [4]:
line = pd.read_excel('dannye.xlsx', sheet_name='line')
trains = pd.read_excel('dannye.xlsx', sheet_name='trains')
pod_sos = pd.read_excel('dannye.xlsx', sheet_name='pod_sos')
vsp_konstr = pd.read_excel('dannye.xlsx', sheet_name='vsp_constr')
damage_koef = pd.read_excel('dannye.xlsx', sheet_name='damage_koeff')

In [5]:
damage_koef

,name,a,b,c,d,e,f,price_rub,gamma,degre_1,degre_2
0,rail_dr,6.981734e-64,-1.178994e-50,9.498787e-38,-2.926301e-25,5.562739e-13,0,51.049875,0.0100,4,4
1,rail_odr,7.757483e-65,-1.309994e-51,1.055421e-38,-3.251445e-26,6.180821e-14,0,51.049875,0.0200,4,4
2,fasten,-2.197690e-86,3.749350e-68,-2.178110e-50,4.859140e-33,-1.272590e-16,0,2.756040,0.0010,4,4
3,tie,-1.248000e-86,2.129400e-68,-1.237100e-50,2.760400e-33,-7.232900e-17,0,5.430000,0.0010,4,4
4,shkol_dop,0.000000e+00,-2.762100e-70,2.325400e-52,4.387900e-35,7.137900e-18,0,0.574945,0.0050,4,4
5,shkol_pred,0.000000e+00,-9.205900e-71,7.751700e-53,1.463000e-35,2.379300e-18,0,0.766594,0.0100,4,4
6,shkol_otkaz,0.000000e+00,-2.301200e-71,1.938200e-53,3.656400e-36,5.948800e-19,0,0.958242,0.0300,4,4
7,prof_dop,-4.000000e-79,1.000000e-62,-1.000000e-46,2.000000e-31,3.000000e-15,0,7.421512,0.0025,4,4
8,prof_pred,-1.227800e-79,4.637500e-63,-4.606300e-47,5.168000e-32,1.141400e-15,0,9.895350,0.0050,4,4
9,prof_otkaz,-4.604000e-80,1.739000e-63,-1.727000e-47,1.938000e-32,4.280000e-16,0,12.369187,0.0150,4,4


In [6]:
# бЛОК вычисления сил для каждого элемента плана с полным учетом вагонопотока на линии
force_line = np.zeros((len(line.index),14))
for index_line in range(len(line.index)):
    #print (index)
    input_val = np.array([0, 0, 0,0,0,0,0,0.00])
    input_val[0]= line['vsp_type'].loc[line['vsp_type'].index == index_line].values
    input_val[1]= line['vsp_cnd'].loc[line['vsp_cnd'].index == index_line].values
    input_val[2]= line['rad_m'].loc[line['rad_m'].index == index_line].values
    input_val[3]= line['h_mm'].loc[line['h_mm'].index == index_line].values
    input_val[6]= line['Shkol_mm'].loc[line['Shkol_mm'].index == index_line].values
    input_val[7]= 0.25 
    force_index = np.array([0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])
    for index_vagon in range(len(pod_sos.index)):
        gamma = pod_sos['gamma'].loc[pod_sos['gamma'].index == index_vagon].values
        p_os = pod_sos['p_os'].loc[pod_sos['p_os'].index == index_vagon].values
        vag_type = pod_sos['vag_type'].loc[pod_sos['vag_type'].index == index_vagon].values  
        meankver_force = pod_sos['meankver_force'].loc[pod_sos['meankver_force'].index == index_vagon].values  
        meankgor_force = pod_sos['meankgor_force'].loc[pod_sos['meankgor_force'].index == index_vagon].values  
        meankprd_force = pod_sos['meankprd_force'].loc[pod_sos['meankprd_force'].index == index_vagon].values  
        meankram_force = pod_sos['meankram_force'].loc[pod_sos['meankram_force'].index == index_vagon].values  
        rmskver_force = pod_sos['rmskver_force'].loc[pod_sos['rmskver_force'].index == index_vagon].values  
        rmskgor_force = pod_sos['rmskgor_force'].loc[pod_sos['rmskgor_force'].index == index_vagon].values  
        rmskprd_force = pod_sos['rmskprd_force'].loc[pod_sos['rmskprd_force'].index == index_vagon].values  
        rmskram_force = pod_sos['rmskram_force'].loc[pod_sos['rmskram_force'].index == index_vagon].values  
        force_v = np.array([0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])
        for v in (10,30,50,70,90,110,130):
            gamma_v_name = 'gamma_' + str(10+v)
            gamma_v = pod_sos[gamma_v_name].loc[pod_sos[gamma_v_name].index == index_vagon].values 
            input_val[4] = v
            input_val[5] = p_os
            #print (input_val)
            #print (vag_type, gamma_v)
            if vag_type == 1:
                force = vagon_1 (input_val, 'return')*gamma_v
                for i in [0,1,2]:
                    force[i] = force[i]*meankver_force
                for i in [3,4,5]:
                    force[i] = force[i]*rmskver_force
                for i in [6,7,8]:
                    force[i] = force[i]*meankgor_force
                for i in [9,10,11]:
                    force[i] = force[i]*rmskgor_force
                force[12] = force[12]*meankram_force
                force[13] = force[13]*rmskram_force
                #print (force)
            elif vag_type == 2:
                force = vagon_2 (input_val, 'return')*gamma_v
                #print (force)
                for i in [0,1,2]:
                    force[i] = force[i]*meankver_force
                for i in [3,4,5]:
                    force[i] = force[i]*rmskver_force
                for i in [6,7,8]:
                    force[i] = force[i]*meankgor_force
                for i in [9,10,11]:
                    force[i] = force[i]*rmskgor_force
                force[12] = force[12]*meankram_force
                force[13] = force[13]*rmskram_force
                
            elif vag_type == 3:
                force = vagon_3 (input_val, 'return')*gamma_v
                #print (force)
                for i in [0,1,2]:
                    force[i] = force[i]*meankver_force
                for i in [3,4,5]:
                    force[i] = force[i]*rmskver_force
                for i in [6,7,8]:
                    force[i] = force[i]*meankgor_force
                for i in [9,10,11]:
                    force[i] = force[i]*rmskgor_force
                force[12] = force[12]*meankram_force
                force[13] = force[13]*rmskram_force
            elif vag_type == 4:
                force = vagon_4 (input_val, 'return')*gamma_v
                #print (force)
                for i in [0,1,2]:
                    force[i] = force[i]*meankver_force
                for i in [3,4,5]:
                    force[i] = force[i]*rmskver_force
                for i in [6,7,8]:
                    force[i] = force[i]*meankgor_force
                for i in [9,10,11]:
                    force[i] = force[i]*rmskgor_force
                force[12] = force[12]*meankram_force
                force[13] = force[13]*rmskram_force
            #else:
                #print ("Номер типа подвижного состава указан неверно или отстуствует в базе")
            force_v = force_v + force
        force_index = force_index +  force_v*gamma 
    for i in range(0,14):
        force_line[index_line, i] = force_index[i]




In [7]:
# бЛОК вычисления сил для каждого элемента плана с полным учетом поездопотока и уклона на линии
force_poezd = np.zeros((len(line.index),14))
aprox_masxi = [-1.8656*(10**-18), 2.4627*(10**-12), 2.0676*(10**-7),1.0262] # коэфф к апроксимации зависимости сил от массы и уклона
for index_line in range(len(line.index)):
    grad = line['i_prm'].loc[line['i_prm'].index == index_line].values 
    
    force_t = np.array([0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])
    
    for index_poezd in range(len(trains.index)):
        massa_poezd = trains['mpoezda_mean'].loc[trains['mpoezda_mean'].index == index_poezd].values
        gamma_poezd = trains['poezd_gamma'].loc[trains['poezd_gamma'].index == index_poezd].values
        k_massxi = (((grad*massa_poezd)**3)*aprox_masxi[0] + ((grad*massa_poezd)**2)*aprox_masxi[1]\
                     + (grad*massa_poezd)*aprox_masxi[2] + aprox_masxi[3])       
        meankver_force = trains['meankver_force'].loc[trains['meankver_force'].index == index_poezd].values  
        meankgor_force = trains['meankgor_force'].loc[trains['meankgor_force'].index == index_poezd].values  
        meankprd_force = trains['meankprd_force'].loc[trains['meankprd_force'].index == index_poezd].values  
        meankram_force = trains['meankram_force'].loc[trains['meankram_force'].index == index_poezd].values  
        rmskver_force = trains['rmskver_force'].loc[trains['rmskver_force'].index == index_poezd].values  
        rmskgor_force = trains['rmskgor_force'].loc[trains['rmskgor_force'].index == index_poezd].values  
        rmskprd_force = trains['rmskprd_force'].loc[trains['rmskprd_force'].index == index_poezd].values  
        rmskram_force = trains['rmskram_force'].loc[trains['rmskram_force'].index == index_poezd].values
        for i in [0,1,2]:
            force[i] = force_line[index_line][i]*meankver_force*gamma_poezd*k_massxi
        for i in [3,4,5]:
            force[i] = force_line[index_line][i]*rmskver_force*gamma_poezd*k_massxi
        for i in [6,7,8]:
            force[i] = force_line[index_line][i]*meankgor_force*gamma_poezd*k_massxi
        for i in [9,10,11]:
            force[i] = force_line[index_line][i]*rmskgor_force*gamma_poezd*k_massxi 
        force[12] = force_line[index_line][12]*meankram_force*gamma_poezd*k_massxi
        force[13] = force_line[index_line][13]*rmskram_force*gamma_poezd*k_massxi 
        
        
        #print(force)
        force_t = force_t + force 
    #print (force_t)
    for i in range(0,14):
        force_poezd[index_line, i] = force_t[i]



    

    

In [8]:
# Формирование итогового массива с силами для каждого элемента
forces_itogo = pd.DataFrame(force_poezd)
forces_itogo.columns = ['Mean_F_vertR_kN', 'Mean_F_vertL_kN', 'Mean_F_vert_kN', 'sigma_F_vertR_kN', 'sigma_F_vertL_kN',\
                        'sigma_F_vert_kN','Mean_F_sideR_kN', 'Mean_F_sideL_kN', 'Mean_F_side_kN', 'sigma_F_sideR_kN',\
                        'sigma_F_sideL_kN', 'sigma_F_side_kN', 'Mean_Hp_kN', 'sigma_Hp_kN' ]


In [9]:
# Формирование файла с силами для каждого элемента
force_line = line.join(forces_itogo)
force_line.to_csv('force_line.csv')
force_line

,km,pk,way,vsp_type,vsp_cnd,rail,wear_mm,tie,fasten,hball_sm,...,sigma_F_vertL_kN,sigma_F_vert_kN,Mean_F_sideR_kN,Mean_F_sideL_kN,Mean_F_side_kN,sigma_F_sideR_kN,sigma_F_sideL_kN,sigma_F_side_kN,Mean_Hp_kN,sigma_Hp_kN
0,1,1,1,1,1,P65,1,beton,kb,20,...,11.721098,11.246846,17.614933,32.043242,24.825475,7.446120,8.785792,8.157756,2.612773,8.497319
1,1,1,1,1,1,P65,1,beton,gbr_bp,30,...,11.660198,11.175318,17.512616,31.859918,24.682676,7.403520,8.735014,8.111085,2.601930,8.448705
2,1,2,1,1,1,P75,1,beton,gbr_pod,40,...,5.054016,4.804334,9.775464,19.584185,14.681871,4.918942,4.732192,4.864196,3.742674,5.476122
3,1,2,1,1,1,P65,2,beton,ars,25,...,11.663276,11.194816,17.512616,31.859918,24.682676,7.404546,8.735014,8.110059,2.626559,8.443574
4,1,3,1,1,1,P65,1,beton,pandr,45,...,11.654040,11.182501,17.514155,31.859918,24.683445,7.403520,8.735528,8.111085,2.597825,8.448705
5,1,3,1,1,1,P65,2,beton,whslo,20,...,2.418011,2.359776,6.709825,12.122520,9.417718,5.469383,6.510900,6.052757,12.136949,6.493894
6,1,4,1,2,2,P65,3,der,do,30,...,12.431387,11.916748,17.598817,31.822975,24.712948,8.029502,9.493376,8.824294,2.475708,9.189621
7,1,4,1,2,2,P75,5,der,kd,40,...,12.442878,11.927763,17.615085,31.852392,24.735793,8.036924,9.502152,8.832451,2.477996,9.198116
8,1,5,1,2,2,P65,9,beton,kb,25,...,7.857100,7.846838,18.136546,16.422279,17.281465,4.642529,6.494820,5.760574,6.451206,6.031490
9,1,5,1,1,1,P65,5,beton,gbr_bp,45,...,11.654040,11.182501,17.514155,31.859918,24.683445,7.403520,8.735528,8.111085,2.597825,8.448705


In [14]:
kv
    kg
    wg
    wv
      f_shp

IndentationError: unexpected indent (<ipython-input-14-562ac0871a74>, line 2)

In [11]:
# бЛОК вычисления напряжений в рельсах для каждого элемента линии с полным учетом поездопотока и уклона на линии
#вычисления ведутся для осредненных значений
rail_streses = np.zeros((len(line.index),7))
tie_forces = np.zeros((len(line.index),10))
balast_opzp_streses = np.zeros((len(line.index),7))

for index_line in range(len(line.index)):
    input_val = np.array([0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])
    rail = line['rail'].loc[line['rail'].index == index_line].values 
    tie = line['tie'].loc[line['tie'].index == index_line].values
    fasten = line['fasten'].loc[line['fasten'].index == index_line].values
    input_val[10] = line['hball_sm'].loc[line['hball_sm'].index == index_line].values
    rad_m = line['rad_m'].loc[line['rad_m'].index == index_line].values
     
    if rad_m > 1200:
        epur = 1840
    else:
        epur = 2000
    ballast = 'sheb'
    proklad = 'tipov'
    input_val[6] = 100000/epur
    input_val[11] = 1
   
    vsp_index = vsp_konstr.index[(vsp_konstr['rail'] == rail[0]) & (vsp_konstr['epur'] == epur)\
                                & (vsp_konstr['tie'] == tie[0]) & (vsp_konstr['fasten'] == fasten[0])].tolist()
    if len(vsp_index) == 0:
        vsp_index = vsp_konstr.index[(vsp_konstr['rail'] == rail[0]) & (vsp_konstr['epur'] == epur)\
                                & (vsp_konstr['tie'] == tie[0]) & (vsp_konstr['fasten'] == 'all')].tolist()
    if len(vsp_index) == 0:
        vsp_index = vsp_konstr.index[(vsp_konstr['rail'] == rail[0]) & (vsp_konstr['epur'] == epur)\
                                & (vsp_konstr['tie'] == tie[0])].tolist()
    if len(vsp_index) == 0:
        print ('В базе отсуствует соответствующая конструкция пути')
        print ('Для расчета будет принята типовая конструкция пути - Р65, жб шпалы, щеб, 1840 шт/км, ЖБР-Ш')
        vsp_index = [4]
    if len(vsp_index) > 1:
        vsp_index = [vsp_index[0]] 
    input_val[4] = vsp_konstr['kg_sm'].loc[vsp_konstr['kg_sm'].index == vsp_index].values 
    input_val[5] = vsp_konstr['kv_sm'].loc[vsp_konstr['kv_sm'].index == vsp_index].values 
    input_val[7] = vsp_konstr['wg_cm3'].loc[vsp_konstr['wg_cm3'].index == vsp_index].values 
    input_val[8] = vsp_konstr['wv_cm3'].loc[vsp_konstr['wv_cm3'].index == vsp_index].values  
    
    input_val[9] = vsp_konstr['omega_sm2'].loc[vsp_konstr['omega_sm2'].index == vsp_index].values 
    force_names = ['Mean_F_vert_kN', 'sigma_F_vert_kN', 'Mean_F_side_kN','sigma_F_side_kN']
    i = 0
    for force_name in force_names:
        input_val[i] = force_line[force_name].loc[force_line[force_name].index == index_line].values
        input_val[12] = force_line['Mean_Hp_kN'].loc[force_line['Mean_Hp_kN'].index == index_line].values
        input_val[13] = force_line['sigma_Hp_kN'].loc[force_line['sigma_Hp_kN'].index == index_line].values
        i = i+1
    
    
    input_val[14] = damage_koef['degre_1'].loc[damage_koef['name'] == 'rail_dr'].values
    input_val[15] = damage_koef['degre_2'].loc[damage_koef['name'] == 'rail_dr'].values
    tensions = krom_tension(input_val)
    for i in range(0,7):
        rail_streses[index_line, i] = tensions[i]
    
    input_val[14] = damage_koef['degre_1'].loc[damage_koef['name'] == 'fasten'].values
    input_val[15] = damage_koef['degre_2'].loc[damage_koef['name'] == 'fasten'].values
    input_val[16] = damage_koef['degre_1'].loc[damage_koef['name'] == 'tie'].values
    input_val[17] = damage_koef['degre_2'].loc[damage_koef['name'] == 'tie'].values
    input_val[18] = damage_koef['degre_1'].loc[damage_koef['name'] == 'shkol_dop'].values
    input_val[19] = damage_koef['degre_2'].loc[damage_koef['name'] == 'shkol_dop'].values
    input_val[20] = damage_koef['degre_1'].loc[damage_koef['name'] == 'plan_dop'].values
    input_val[21] = damage_koef['degre_2'].loc[damage_koef['name'] == 'plan_dop'].values
    tie_force =  f_shpal(input_val) 
    for i in range(0,10):
        tie_forces[index_line, i] = tie_force[i]
    
    
    input_val[14] = damage_koef['degre_1'].loc[damage_koef['name'] == 'prof_dop'].values
    input_val[15] = damage_koef['degre_2'].loc[damage_koef['name'] == 'prof_dop'].values
    input_val[16] = damage_koef['degre_1'].loc[damage_koef['name'] == 'ballast'].values
    input_val[17] = damage_koef['degre_2'].loc[damage_koef['name'] == 'ballast'].values
    input_val[18] = damage_koef['degre_1'].loc[damage_koef['name'] == 'opzp'].values
    input_val[19] = damage_koef['degre_2'].loc[damage_koef['name'] == 'opzp'].values
    balast_opzp_stress = f_ballast_opzp(input_val) 
    for i in range(0,7):
        balast_opzp_streses[index_line, i] = balast_opzp_stress[i] 
    
   
        
   



In [12]:
rail_streses_itogo = pd.DataFrame(rail_streses)
rail_streses_itogo.columns = ['Mean_nar_krom_MPa', 'RMS_nar_krom_MPa', 'Mean_vntr_krom_MPa', 'RMS_vntr_krom_MPa',\
                             'Mean_osev_MPa', 'RMS_osev_MPa','d_rail_MPa^Xrail']

rail_streses = line.join(rail_streses_itogo)
rail_streses.to_csv('rail_streses_line.csv')
rail_streses 

,km,pk,way,vsp_type,vsp_cnd,rail,wear_mm,tie,fasten,hball_sm,...,i_prm,l_m,Shkol_mm,Mean_nar_krom_MPa,RMS_nar_krom_MPa,Mean_vntr_krom_MPa,RMS_vntr_krom_MPa,Mean_osev_MPa,RMS_osev_MPa,d_rail_MPa^Xrail
0,1,1,1,1,1,P65,1,beton,kb,20,...,5,50,1520,51.5,6.1,30.9,6.1,41.2,5.0,2.956066e+06
1,1,1,1,1,1,P65,1,beton,gbr_bp,30,...,0,50,1525,51.4,6.1,30.8,6.1,41.1,5.0,2.927887e+06
2,1,2,1,1,1,P75,1,beton,gbr_pod,40,...,-5,50,1515,37.9,2.3,28.3,2.3,33.1,1.7,1.206695e+06
3,1,2,1,1,1,P65,2,beton,ars,25,...,0,50,1530,46.1,5.4,27.7,5.4,36.9,4.5,1.901063e+06
4,1,3,1,1,1,P65,1,beton,pandr,45,...,0,50,1520,51.2,6.0,30.7,6.0,41.0,5.0,2.897639e+06
5,1,3,1,1,1,P65,2,beton,whslo,20,...,4,50,1520,49.7,2.7,41.8,2.7,45.8,1.1,4.415642e+06
6,1,4,1,2,2,P65,3,der,do,30,...,0,50,1520,96.2,16.4,13.8,16.4,55.0,7.1,1.286362e+07
7,1,4,1,2,2,P75,5,der,kd,40,...,1,50,1520,86.9,14.8,12.7,14.8,49.8,6.5,8.600325e+06
8,1,5,1,2,2,P65,9,beton,kb,25,...,0,50,1520,48.1,4.0,34.5,4.0,41.3,3.3,2.938872e+06
9,1,5,1,1,1,P65,5,beton,gbr_bp,45,...,0,50,1520,51.4,6.1,30.8,6.1,41.1,5.0,2.927887e+06


In [13]:
tie_forces_itogo = pd.DataFrame(tie_forces)
tie_forces_itogo.columns = ['mean_F_shpal_vert_kN','sigma_F_shpal_vert_kN',\
                            'mean_F_shpal_side_kN', 'sigma_F_shpal_side_kN',\
                           'mean_Hp_kN', 'sigma_Hp_kN',\
                            'd_fast_kN^Xfast', 'd_tie_kN^Xtie', 'd_shkol_kN^Xshkol', 'd_plan_kN^Xplan']

tie_forces = line.join(tie_forces_itogo)
tie_forces.to_csv('tie_forces_line.csv')
tie_forces 


,km,pk,way,vsp_type,vsp_cnd,rail,wear_mm,tie,fasten,hball_sm,...,mean_F_shpal_vert_kN,sigma_F_shpal_vert_kN,mean_F_shpal_side_kN,sigma_F_shpal_side_kN,mean_Hp_kN,sigma_Hp_kN,d_fast_kN^Xfast,d_tie_kN^Xtie,d_shkol_kN^Xshkol,d_plan_kN^Xplan
0,1,1,1,1,1,P65,1,beton,kb,20,...,33.4,4.1,54.2,17.8,5.7,18.5,1.187390e+07,1.187390e+07,1.187390e+07,0.109517
1,1,1,1,1,1,P65,1,beton,gbr_bp,30,...,33.1,4.1,53.7,17.6,5.7,18.4,1.143560e+07,1.143560e+07,1.143560e+07,0.111257
2,1,2,1,1,1,P75,1,beton,gbr_pod,40,...,36.6,1.9,34.4,11.4,8.8,12.8,3.528915e+06,3.528915e+06,3.528915e+06,0.032269
3,1,2,1,1,1,P65,2,beton,ars,25,...,36.9,4.5,59.8,19.6,6.4,20.5,1.759272e+07,1.759272e+07,1.759272e+07,0.111396
4,1,3,1,1,1,P65,1,beton,pandr,45,...,33.2,4.1,53.8,17.7,5.7,18.4,1.154180e+07,1.154180e+07,1.154180e+07,0.109943
5,1,3,1,1,1,P65,2,beton,whslo,20,...,37.1,0.9,20.5,13.2,26.5,14.2,2.250153e+06,2.250153e+06,2.250153e+06,0.430749
6,1,4,1,2,2,P65,3,der,do,30,...,24.9,3.2,13.4,4.8,1.3,5.0,4.382618e+05,4.382618e+05,4.382618e+05,0.001793
7,1,4,1,2,2,P75,5,der,kd,40,...,23.3,3.0,12.6,4.5,1.3,4.7,3.366272e+05,3.366272e+05,3.366272e+05,0.001857
8,1,5,1,2,2,P65,9,beton,kb,25,...,34.8,2.8,36.8,12.3,13.8,12.9,3.752289e+06,3.752289e+06,3.752289e+06,0.085714
9,1,5,1,1,1,P65,5,beton,gbr_bp,45,...,33.1,4.1,53.7,17.6,5.7,18.4,1.143560e+07,1.143560e+07,1.143560e+07,0.111257


In [14]:
balast_opzp_streses_itogo = pd.DataFrame(balast_opzp_streses)
balast_opzp_streses_itogo.columns = ['mean_F_ballast_kPa', 'sigma_F_ballast_kPa', 'mean_F_OPZP_kPa', 'sigma_F_OPZP_kPa',\
                                    'd_prof_kPa^Xprof', 'd_ball_kPa^Xball', 'd_opzp_kPa^Xopzp']

balast_opzp_streses = line.join(balast_opzp_streses_itogo)
balast_opzp_streses.to_csv('balast_opzp_streses_line.csv')
balast_opzp_streses 

,km,pk,way,vsp_type,vsp_cnd,rail,wear_mm,tie,fasten,hball_sm,...,i_prm,l_m,Shkol_mm,mean_F_ballast_kPa,sigma_F_ballast_kPa,mean_F_OPZP_kPa,sigma_F_OPZP_kPa,d_prof_kPa^Xprof,d_ball_kPa^Xball,d_opzp_kPa^Xopzp
0,1,1,1,1,1,P65,1,beton,kb,20,...,5,50,1520,108.1,13.2,50.8,3.7,1.406560e+08,3.146706e+07,1.688463e+07
1,1,1,1,1,1,P65,1,beton,gbr_bp,30,...,0,50,1525,107.2,13.1,49.5,3.6,1.360361e+08,2.430155e+07,1.112862e+07
2,1,2,1,1,1,P75,1,beton,gbr_pod,40,...,-5,50,1515,118.4,6.1,50.6,1.9,1.975647e+08,1.847489e+07,6.507511e+06
3,1,2,1,1,1,P65,2,beton,ars,25,...,0,50,1530,119.4,14.6,49.9,3.7,2.093670e+08,7.080429e+07,1.301101e+07
4,1,3,1,1,1,P65,1,beton,pandr,45,...,0,50,1520,107.5,13.1,48.3,3.7,1.375427e+08,2.171170e+07,8.235393e+06
5,1,3,1,1,1,P65,2,beton,whslo,20,...,4,50,1520,120.1,2.8,60.2,1.3,2.082783e+08,8.274193e+07,7.451176e+07
6,1,4,1,2,2,P65,3,der,do,30,...,0,50,1520,87.3,11.4,49.7,3.9,6.008188e+07,1.542330e+07,1.390695e+07
7,1,4,1,2,2,P75,5,der,kd,40,...,1,50,1520,81.8,10.6,49.0,3.9,4.628897e+07,1.217662e+07,1.136231e+07
8,1,5,1,2,2,P65,9,beton,kb,25,...,0,50,1520,112.5,9.0,55.8,2.7,1.622375e+08,5.395122e+07,4.287637e+07
9,1,5,1,1,1,P65,5,beton,gbr_bp,45,...,0,50,1520,107.2,13.1,48.3,3.7,1.360361e+08,2.140832e+07,8.235393e+06


In [23]:
damage_itogo = pd.DataFrame()
damage_itogo['d_rail_MPa^Xrail'] = rail_streses['d_rail_MPa^Xrail']
damage_itogo['d_fast_kN^Xfast'] = tie_forces['d_fast_kN^Xfast']
damage_itogo['d_tie_kN^Xtie'] = tie_forces['d_tie_kN^Xtie']
damage_itogo['d_shkol_kN^Xshkol'] = tie_forces['d_shkol_kN^Xshkol']
damage_itogo['d_prof_kPa^Xprof'] = balast_opzp_streses['d_prof_kPa^Xprof']
damage_itogo['d_plan_kN^Xplan'] = tie_forces['d_plan_kN^Xplan']
damage_itogo['d_ball_kPa^Xball'] = balast_opzp_streses['d_ball_kPa^Xball']
damage_itogo['d_opzp_kPa^Xopzp'] = balast_opzp_streses['d_opzp_kPa^Xopzp']
damage = line.join(damage_itogo)
damage.to_csv('damage_line.csv')
damage

,km,pk,way,vsp_type,vsp_cnd,rail,wear_mm,tie,fasten,hball_sm,...,l_m,Shkol_mm,d_rail_MPa^Xrail,d_fast_kN^Xfast,d_tie_kN^Xtie,d_shkol_kN^Xshkol,d_prof_kPa^Xprof,d_plan_kN^Xplan,d_ball_kPa^Xball,d_opzp_kPa^Xopzp
0,1,1,1,1,1,P65,1,beton,kb,20,...,50,1520,2.956066e+06,1.187390e+07,1.187390e+07,1.187390e+07,1.406560e+08,0.109517,3.146706e+07,1.688463e+07
1,1,1,1,1,1,P65,1,beton,gbr_bp,30,...,50,1525,2.927887e+06,1.143560e+07,1.143560e+07,1.143560e+07,1.360361e+08,0.111257,2.430155e+07,1.112862e+07
2,1,2,1,1,1,P75,1,beton,gbr_pod,40,...,50,1515,1.206695e+06,3.528915e+06,3.528915e+06,3.528915e+06,1.975647e+08,0.032269,1.847489e+07,6.507511e+06
3,1,2,1,1,1,P65,2,beton,ars,25,...,50,1530,1.901063e+06,1.759272e+07,1.759272e+07,1.759272e+07,2.093670e+08,0.111396,7.080429e+07,1.301101e+07
4,1,3,1,1,1,P65,1,beton,pandr,45,...,50,1520,2.897639e+06,1.154180e+07,1.154180e+07,1.154180e+07,1.375427e+08,0.109943,2.171170e+07,8.235393e+06
5,1,3,1,1,1,P65,2,beton,whslo,20,...,50,1520,4.415642e+06,2.250153e+06,2.250153e+06,2.250153e+06,2.082783e+08,0.430749,8.274193e+07,7.451176e+07
6,1,4,1,2,2,P65,3,der,do,30,...,50,1520,1.286362e+07,4.382618e+05,4.382618e+05,4.382618e+05,6.008188e+07,0.001793,1.542330e+07,1.390695e+07
7,1,4,1,2,2,P75,5,der,kd,40,...,50,1520,8.600325e+06,3.366272e+05,3.366272e+05,3.366272e+05,4.628897e+07,0.001857,1.217662e+07,1.136231e+07
8,1,5,1,2,2,P65,9,beton,kb,25,...,50,1520,2.938872e+06,3.752289e+06,3.752289e+06,3.752289e+06,1.622375e+08,0.085714,5.395122e+07,4.287637e+07
9,1,5,1,1,1,P65,5,beton,gbr_bp,45,...,50,1520,2.927887e+06,1.143560e+07,1.143560e+07,1.143560e+07,1.360361e+08,0.111257,2.140832e+07,8.235393e+06


In [18]:
['Mean_F_vert_kN', 'sigma_F_vert_kN', 'Mean_F_side_kN','sigma_F_side_kN']
    q_mean = input_val[0] 
    q_rms = input_val[1] 
    Y_mean = input_val[2] 
    Y_rms = input_val[3] 
    Kg = input_val[4] 
    Kv = input_val[5] 
    l_sh = input_val[6]
    Wg = input_val[7] 
    Wv = input_val[8]
    f_shp = input_val[9]
    h_ballast = input_val[10]
    ballast_type = input_val[11]
    
def f_shpal(input):

    '''
       Функция вычисления сил, передающихся на шпалы от рельса
       Исходные данные:
       Q_mean/rms, кН - вертикальная сила (среднее/СКО)
       Y_mean/rms, кН - боковая сила (среднее/СКО)
       Kg, cм^-1 - коэффициент отностительной жесткости рельсовой нити в гориз.плоскости
       Kv, cм^-1 - коэффициент отностительной жесткости рельсовой нити в вертик.плоскости
       l_shp,см - расстояние между осями шпал
       Выходные данные:
       (mean/sigma) F_shpal_vert/side, кН - среднее значение/СКО сил действующая от рельса на шпалу в вертикальной/горизонтальной плоскости
    '''
    Q_mean = input[0]
    Q_rms = input[1]
    Y_mean = input[2]
    Y_rms = input[3]
    Kg = input[4]
    Kv = input[5]
    l_sh = pinput[6] 


IndentationError: unexpected indent (<ipython-input-18-1d4f4195d374>, line 2)

In [ ]:
In [5]: df1.merge(df2) # by default, it does an inner join on the common column(s)
Out[5]: 
   x  y  z
0  2  b  4
1  3  c  5

df2 = df1.copy()
with pd.ExcelWriter('output.xlsx') as writer:  
    df1.to_excel(writer, sheet_name='Sheet_name_1')
    df2.to_excel(writer, sheet_name='Sheet_name_2')
    
#ExcelWriter также можно использовать для добавления в существующий файл Excel:

with pd.ExcelWriter('output.xlsx',
                    mode='a') as writer:  
    df.to_excel(writer, sheet_name='Sheet_name_3')
    
b[:2]  # Первые две строки
array([[ 0,  1,  2,  3],
       [10, 11, 12, 13]])
>>> b[1:3, : : ]  # Вторая и третья строки
array([[10, 11, 12, 13],
       [20, 21, 22, 23]])